In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os
import math
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from operator import itemgetter

from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

#Importing three component ensembles
from sklearn.ensemble import VotingClassifier

#importing SVC for second-step classification
from sklearn.svm import SVC

In [2]:
mutationRate = 0.001
crossOverRate = 0.06
iterations = 10
poolSize = 50

In [3]:
#Defining functions for genetic algorithms

def roulette(fitnessArray):
    index = 0
    cumalativeFitness = 0.0
    r = random.random()
    
    for i in range(len(fitnessArray)):
        cumalativeFitness += fitnessArray[i]
        if cumalativeFitness > r:
            return i


def selectFittest(fitness, rankedPool):
    while True:
        idx1 = roulette(fitness)
        idx2 = roulette(fitness)
        
        if idx1 is None or idx2 is None:
            continue
        elif idx1==idx2:
            continue
        else:
            break
    
    return rankedPool[idx1], rankedPool[idx2]

def crossover(chromosome1, chromosome2):
    randomSplitPoint = random.randint(1, len(chromosome1))
    return np.concatenate((chromosome1[:randomSplitPoint],chromosome2[randomSplitPoint:])), np.concatenate((chromosome2[:randomSplitPoint],chromosome1[randomSplitPoint:]))


def mutate(chromosome):

    mutatedChrom = []
    for ch in chromosome:
        if random.random()<mutationRate:
            if ch==1:
                mutatedChrom.append(0)
            else:
                mutatedChrom.append(1)
        else:
            mutatedChrom.append(ch)
    return mutatedChrom
    
def breed(chrome1, chrome2):
    if random.random()<crossOverRate:
        newChrome1, newChrome2 = crossover(chrome1, chrome2)
    else:
        newChrome1 = chrome1
        newChrome2 = chrome2
        
    newChrome1 = mutate(newChrome1)
    newChrome2 = mutate(newChrome2)
    
    return newChrome1, newChrome2

In [4]:
def rankPop(pool, X, y, classifier,fitnessFunction):
    scores = []
    for chromosome in pool:
        classifier = RandomForestClassifier()
        chosen_idx = [idx for gene, idx in zip(chromosome, range(X.shape[1])) if gene==1]
        if len(chosen_idx)==0:
            continue
        chosenX = X.iloc[:, chosen_idx]
        #performing leave-one out validation for instances less than 100
        #and 10 fold validation for others
        npoints = X.shape[0]
   
        if npoints <= 100:
            kf = KFold(n_splits = npoints)
        else:
            kf = KFold(n_splits = 10)
        
        kf.get_n_splits(X)
        classifier.fit(chosenX, y)
        train_X = []
        train_Y  = []
        prediction   = []
        predict_prob = []
        chosenX = np.array(chosenX)
        Y = np.array(y)
        for train_index, test_index in kf.split(X):
            classifier = RandomForestClassifier()
            for i in train_index:
                train_X.append(chosenX[i])
                train_Y.append(Y[i])

            classifier.fit(train_X, train_Y)
            for j in test_index:
                prediction.append(classifier.predict([chosenX[j]])[0])
                predict_prob.append(classifier.predict_proba([chosenX[j]])[0][1])
            train_X  = []
            train_Y  = []
        
   
        if(fitnessFunction == 'f-measure'):
            scores.append(f1_score(y_true=y,y_pred=prediction))
        elif(fitnessFunction == 'g-mean'):
            gScore = math.sqrt(precision_score(y_true = y, y_pred=prediction)*recall_score(y_true = y, y_pred =prediction ))
            scores.append(gScore)
        elif(fitnessFunction == 'accuracy'):
            scores.append(accuracy_score(y_true = y, y_pred = prediction))
        
    fitness = [x/sum(scores) for x in scores]
    pairedPop = zip(pool, fitness)
    rankedPop = sorted(pairedPop, key=itemgetter(-1), reverse = True)
    
    return rankedPop

In [5]:
def iteratePop(rankedPop):
    fitness = [item[-1] for item in rankedPop]
    rankedPool = [item[0] for item in rankedPop]
   
    new_pool = []
    new_pool.extend(rankedPool[:int(poolSize/15)])
    
    while(len(new_pool)<poolSize):
        ch1, ch2 = selectFittest(fitness, rankedPool)
        ch1, ch2 = breed(ch1, ch2)
        
        new_pool.append(ch1)
        new_pool.append(ch2)
    
    return new_pool[:poolSize]

In [6]:
def geneticAlgoFit(datafile,measure):
    datafile = pd.read_csv(datafile, dtype={'buggy':np.bool})
    X     = datafile.iloc[ : , :-1]
    y = datafile['buggy']

    pool = np.random.randint(0, 2, (poolSize, X.shape[1]))  
    for iteration in range(iterations):
#         print iteration
        classifier = RandomForestClassifier()
        rankedPop = rankPop(pool, X, y, classifier,measure)
#         print rankedPop
        pool = []
        pool = iteratePop(rankedPop)
        
    best_chromosome = rankPop(pool, X, y, classifier, measure)[0][0]
    return best_chromosome

In [8]:
directory = 'dataset/dataset/'
for fileName in os.listdir(directory):
    print(fileName)
    print(geneticAlgoFit(directory+fileName,'accuracy'))
    print(geneticAlgoFit(directory+fileName,'f-measure'))
    print(geneticAlgoFit(directory+fileName,'g-mean'))

camel-1.6.csv


KeyboardInterrupt: 

In [10]:
selectedFeaturesInEachSoftware = np.array([
    [[0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1]],
    
    [[1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]],
    
    [[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]],
    
    [[0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
    [1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
    [1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]],
    
    [[1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]],
    
    [[0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0]],
    
    [[0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1],
    [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0]],
    
    [[1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]],
    
    [[1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1],
    [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
    
    [[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
    [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0],
    [0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0]],
    
    [[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
    [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]],
    
    [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0],
    [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]],
    
    [[0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0]],
    
    [[1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1],
    [0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1]],
    
    [[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0],
    [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0]],
    
    [[0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]],
    
    [[0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
    [1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1],
    [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1]],
    
    [[0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]],
    
    [[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1],
    [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0],
    [1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]],
    
    [[1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1]],
    
    [[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0]],
    
    [[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0],
    [0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
    [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1]],
    
    [[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1]],
    
    [[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
    [1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]]
])      

In [ ]:
base_learner1 = LogisticRegression(random_state=1)
base_learner2 = DecisionTreeClassifier()
base_learner3 = GaussianNB()
classifier1 = VotingClassifier(estimators=[
                                         ('logregression', base_learner1), 
                                         ('dtree', base_learner2), 
                                         ('gnb', base_learner3)], 
                                          voting='soft')  
classifier2 = RandomForestClassifier()
classifier3 = AdaBoostClassifier(base_estimator = RandomForestClassifier(), n_estimators = 100, learning_rate = 0.5)
directory='dataset/dataset'
for fileName in os.listdir(directory):
    print fileName
    

In [19]:
index = 0
directory = 'dataset/dataset/'
for fileName in os.listdir(directory):
    print fileName
    data = pd.read_csv(directory+fileName)
    X  = data.iloc[ : , :-1]
    Y = data.iloc[:,-1]
    accuracyFeature = selectedFeaturesInEachSoftware[index][0]
    fMeasureFeature = selectedFeaturesInEachSoftware[index][1]
    gMeanFeature = selectedFeaturesInEachSoftware[index][2]
   
    chosen_idx_accuracy = [idx for gene, idx in zip(accuracyFeature, range(X.shape[1])) if gene==1]
    chosen_idx_fMeasure = [idx for gene, idx in zip(fMeasureFeature, range(X.shape[1])) if gene==1]
    chosen_idx_gMean = [idx for gene, idx in zip(gMeanFeature, range(X.shape[1])) if gene==1]
    
    for strings in ['Voting','RF','Ada']:
        print strings
        data['Accuracy_'+strings+'_Pred'],data['Accuracy_'+strings+'_Pred_Prob'] = getPredictions(X.iloc[:,chosen_idx_accuracy],Y,strings)
        data['FMeasure_'+strings+'_Pred'],data['FMeasure_'+strings+'_Pred_Prob'] = getPredictions(X.iloc[:,chosen_idx_fMeasure],Y,strings)   
        data['GMean_'+strings+'_Pred'],data['GMean_'+strings+'_Pred_Prob'] = getPredictions(X.iloc[:,chosen_idx_gMean],Y,strings)
    
    print data['GMean_Ada_Pred']
    data.to_csv('dataset/annotated/'+fileName,index=False)    
    index+=1
    

camel-1.6.csv
Voting
RF
Ada
0      False
1      False
2      False
3      False
4       True
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22      True
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
935    False
936    False
937    False
938    False
939    False
940    False
941    False
942     True
943    False
944    False
945    False
946    False
947    False
948    False
949    False
950    False
951     True
952    False
953    False
954    False
955    False
956    False
957    False
958    False
959    False
960    False
961    False
962    False
963    False
964    False
Name: GMean_Ada_Pred, Length: 965, dtype: bool
e-learning.csv
Voting
RF
Ada
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     F

RF
Ada
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8      True
9     False
10    False
11    False
12    False
13     True
14    False
15     True
16    False
17    False
18    False
19    False
20    False
21     True
22     True
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32     True
33    False
34    False
35    False
36    False
37    False
38     True
39    False
40     True
41    False
42    False
43    False
44    False
Name: GMean_Ada_Pred, dtype: bool
sklebagd.csv
Voting
RF
Ada
0      True
1     False
2      True
3     False
4     False
5      True
6      True
7      True
8     False
9      True
10     True
11     True
12     True
13    False
14     True
15    False
16     True
17     True
18     True
19     True
Name: GMean_Ada_Pred, dtype: bool
synapse-1.2.csv
Voting
RF
Ada
0       True
1       True
2       True
3      False
4      False
5      False
6      Fal

In [18]:
def getPredictions(X,Y,strings):
    
    base_learner1 = LogisticRegression(random_state=1)
    base_learner2 = DecisionTreeClassifier()
    base_learner3 = GaussianNB()

    classifier = VotingClassifier(estimators=[
                                 ('logregression', base_learner1), 
                                 ('dtree', base_learner2), 
                                 ('gnb', base_learner3)], 
                                  voting='soft')
    npoints = X.shape[0]
   
    if npoints <= 100:
        kf = KFold(n_splits = npoints)
    else:
        kf = KFold(n_splits = 10)
        
    kf.get_n_splits(X)
    train_X = []
    train_Y  = []
    prediction   = []
    predict_prob = []
    X = np.array(X)
    Y = np.array(Y)
    for train_index, test_index in kf.split(X):
        if strings == 'RF':
            classifier = RandomForestClassifier()
        elif strings == 'Ada':
            classifier = AdaBoostClassifier()
        else:
            classifier = VotingClassifier(estimators=[
                                 ('logregression', base_learner1), 
                                 ('dtree', base_learner2), 
                                 ('gnb', base_learner3)], 
                                  voting='soft')
        for i in train_index:
            train_X.append(X[i])
            train_Y.append(Y[i])

        classifier.fit(train_X, train_Y)
        for j in test_index:
            prediction.append(classifier.predict([X[j]])[0])
            predict_prob.append(classifier.predict_proba([X[j]])[0][1])
        train_X  = []
        train_Y  = []

    return prediction,predict_prob

In [24]:
def getSelectedFitnessFunction():
    for fileName in os.listdir('dataset/annotated'):
        selectedFitnessFunction = [] 
        data = pd.read_csv('dataset/annotated/'+fileName)
        
        tarr =tt
        arr0,arr1 = data[tarr[0]] , data[tarr[1]]
        arr2,arr3 = data[tarr[2]] , data[tarr[3]]        
        arr4,arr5 = data[tarr[4]] , data[tarr[5]]
        arr6,arr7 = data[tarr[6]] , data[tarr[7]]
        arr8,arr9 = data[tarr[8]] , data[tarr[9]]
        arr10,arr11 = data[tarr[10]] , data[tarr[11]]
        arr12,arr13 = data[tarr[12]] , data[tarr[13]]
        arr14,arr15 = data[tarr[14]] , data[tarr[15]]
        arr16,arr17 = data[tarr[16]] , data[tarr[17]]
        
        auc_arr = []
        auc_arr.append(roc_auc_score(y_true=data['buggy'],y_score=arr1))
        auc_arr.append(roc_auc_score(y_true=data['buggy'],y_score=arr3))
        auc_arr.append(roc_auc_score(y_true=data['buggy'],y_score=arr5))
        auc_arr.append(roc_auc_score(y_true=data['buggy'],y_score=arr7))
        auc_arr.append(roc_auc_score(y_true=data['buggy'],y_score=arr9))
        auc_arr.append(roc_auc_score(y_true=data['buggy'],y_score=arr11))
        auc_arr.append(roc_auc_score(y_true=data['buggy'],y_score=arr13))
        auc_arr.append(roc_auc_score(y_true=data['buggy'],y_score=arr15))
        auc_arr.append(roc_auc_score(y_true=data['buggy'],y_score=arr17))
        
        result = np.where(auc_arr == np.amax(auc_arr))
        
        ans_array  = ['Accuracy_Voting_Pred','FMeasure_Voting_Pred','GMean_Voting_Pred',
                      'Accuracy_RF_Pred','FMeasure_RF_Pred','GMean_RF_Pred','Accuracy_Ada_Pred',
                     'FMeasure_Ada_Pred','GMean_Ada_Pred']
        highest = ans_array[result[0][0]]
        
        buggy = np.array(data['buggy'])
        a = ""
        b=0
        for i in range(len(arr0)):
            flag = True
            count=0
#             print i,
            answer = -1
            if arr0[i]==buggy[i]:
                answer = 0
                count+=1
                if auc_arr[0]>b:
                    a=ans_array[0]
                    b=auc_arr[0]
                    
            if arr2[i]==buggy[i]:
                answer = 2
                count+=1
                if auc_arr[1]>b:
                    a=ans_array[1]
                    b=auc_arr[1]
                    
            if arr4[i]==buggy[i]:
                answer = 4
                count+=1
                if auc_arr[2]>b:
                    a=ans_array[2]
                    b=auc_arr[2]
                
            if arr6[i]==buggy[i]:
                answer = 6
                count+=1
                if auc_arr[3]>b:
                    a=ans_array[3]
                    b=auc_arr[3]
                
            if arr8[i]==buggy[i]:
                answer = 8
                count+=1
                if auc_arr[4]>b:
                    a=ans_array[4]
                    b=auc_arr[4]
                
            if arr10[i]==buggy[i]:
                answer = 10
                count+=1
                if auc_arr[5]>b:
                    a=ans_array[5]
                    b=auc_arr[5]
                
            if arr12[i]==buggy[i]:
                answer = 12
                count+=1
                if auc_arr[6]>b:
                    a=ans_array[6]
                    b=auc_arr[6]
                
            if arr14[i]==buggy[i]:
                answer = 14
                count+=1
                if auc_arr[7]>b:
                    a=ans_array[7]
                    b=auc_arr[7]
                
            if arr16[i]==buggy[i]:
                answer = 16
                count+=1
                if auc_arr[8]>b:
                    a=ans_array[8]
                    b=auc_arr[8]
#             print ("***",answer)
            if count==1 and answer!=-1:
                selectedFitnessFunction.append(ans_array[int(answer/2)])
            else:
                if answer == -1:
                    selectedFitnessFunction.append(highest)
                else:
                    selectedFitnessFunction.append(a)
        print (len(selectedFitnessFunction),len(arr0))
        data['Function to be selected'] = selectedFitnessFunction
#         print data.head()
        data.to_csv('dataset/annotated/'+fileName,index=False)


            

In [25]:
getSelectedFitnessFunction()

965 965
20 20
342 342
588 588
920 920
25 25
229 229
176 176
64 64
205 205
27 27
492 492
24 24
340 340
39 39
45 45
29 29
65 65
522 522
33 33
660 660
858 858
392 392
55 55


In [26]:
def svctrain():
    directory = 'dataset/dataset/'
    annotated_directory = 'dataset/annotated/'
    DSF_directory = 'dataset/DSF/'
    for projectName in os.listdir(directory):
        print(projectName)
        projectData = pd.read_csv(directory + projectName)
        annotatedData = pd.read_csv(annotated_directory + projectName)
        
        #X contains software metrics and Y best ensemble selected
        X = np.array(projectData.iloc[ : , :-1])
        Y = np.array(annotatedData.iloc[ : , -1])
#         print (X[0],X[1],Y[0],Y[1])
        npoints = X.shape[0]
        
        if npoints <= 100:
            kf = KFold(n_splits = npoints)
        else:
            kf = KFold(n_splits = 10)
        
        kf.get_n_splits(X)
        train_X = []
        train_Y = []
        
        predictedEnsemble = []
        predict_prob      = []
        final_prediction  = []  
        
        prediction_constant = '_Pred'
        probab_constant = '_Pred_Prob'
        
        
        for train_index, test_index in kf.split(X):
            classifier = DecisionTreeClassifier()
            
            for i in train_index:
                train_X.append(X[i])
                train_Y.append(Y[i])
            
            unique_labels = np.unique(train_Y)
            if unique_labels.size == 1:
                for j in test_index:
                    predictedEnsemble.append(unique_labels[0])
                    predict_prob.append(annotatedData.loc[j, unique_labels[0]+"_Prob"])
                    final_prediction.append(annotatedData.loc[j, (unique_labels[0])])
           
            else:
                classifier.fit(train_X, train_Y)
                
                for j in test_index:
                    predictedBestEnsemble = classifier.predict([X[j]])[0]
                    predictedEnsemble.append(predictedBestEnsemble)
                    final_prediction.append(annotatedData.loc[j, predictedBestEnsemble])
                    
            # total probability of available classifiers, i.e the classifiers reported in unique_labels predicting true
                    predict_proba_true = 0
                    
            # probability of classifiers being predicted
#                     predict_proba_classifiers = 1#classifier.predict_proba([X[j]])[0]
#                     k = 0
            # class probabilities are always reported in a sorted by name fashion, i.e AdaBoost, RandomForest, Voting 
            # np.unique also reports labels in a sorted by name fashion
#                     for classifierName in unique_labels:
#                         predict_proba_true = 
#                         k += 1
                    predict_prob.append(annotatedData.loc[j, predictedBestEnsemble+"_Prob"])
                    
        annotatedData['PredictedTechnique'] = predictedEnsemble
        annotatedData['DSF_Prediction'] = final_prediction
        annotatedData['DSF_Pred_Prob'] = predict_prob
        annotatedData.to_csv(DSF_directory + projectName, index = False)
        

In [27]:
svctrain()

camel-1.6.csv
sklebagd.csv
synapse-1.2.csv
xerces-1.4.csv
xalan-2.7.csv
szybkafucha.csv
velocity-1.6.csv
redaktor.csv
e-learning.csv
log4j-1.2.csv
intercafe.csv
jedit-4.3.csv
wspomaganiepi.csv
lucene-2.4.csv
workflow.csv
serapion.csv
zuzel.csv
systemdata.csv
poi-2.5.csv
kalkulator.csv
prop-6.csv
tomcat.csv
ivy-2.0.csv
termoproject.csv


In [34]:
def computePerformanceMeasuresDSF():
    DSFdirectory = 'dataset/finalResults/'
    projectMetrics = []
    index = 0
    projectMetrics = pd.DataFrame(projectMetrics,
                                    columns = ['Project','Precision', 'Recall', 'Auc_Score', 'Accuracy', 'Fmeasure', 'GMean'])
    for projectName in os.listdir(DSFdirectory):
        print (projectName)
        project = pd.read_csv(DSFdirectory + projectName)
        projectData = project.as_matrix(columns=[
                                         'DSF_Prediction',
                                         'DSF_Pred_Prob',
                                         'buggy'])
      
        row = []
        row.append(projectName)
        row.extend(computePerformanceMeasures(project['DSF_Prediction'], 
                                                         project['buggy'], 
                                                         project['DSF_Pred_Prob']))
        projectMetrics.loc[index] = row
        index = index + 1
  
    print(projectMetrics)
    projectMetrics.to_csv('dataset/' + 'results.csv', index = False)

In [35]:
computePerformanceMeasuresDSF()

camel-1.6.csv
sklebagd.csv
synapse-1.2.csv
xerces-1.4.csv
xalan-2.7.csv
szybkafucha.csv
velocity-1.6.csv
redaktor.csv
e-learning.csv
log4j-1.2.csv
intercafe.csv
jedit-4.3.csv
wspomaganiepi.csv
lucene-2.4.csv
workflow.csv
serapion.csv
zuzel.csv
systemdata.csv
poi-2.5.csv
kalkulator.csv
prop-6.csv
tomcat.csv
ivy-2.0.csv
termoproject.csv
              Project  Precision    Recall  Auc_Score  Accuracy  Fmeasure  \
0       camel-1.6.csv   0.641975  0.276596   0.745249  0.829016  0.386617   
1        sklebagd.csv   0.846154  0.916667   0.916667  0.850000  0.880000   
2     synapse-1.2.csv   0.853659  0.813953   0.895982  0.836257  0.833333   
3      xerces-1.4.csv   0.957111  0.970252   0.966160  0.945578  0.963636   
4       xalan-2.7.csv   0.993318  0.993318   0.978538  0.986957  0.993318   
5     szybkafucha.csv   0.750000  0.642857   0.844156  0.680000  0.692308   
6    velocity-1.6.csv   0.705882  0.615385   0.795976  0.781659  0.657534   
7        redaktor.csv   0.687500  0.407407   0.

In [1]:
def computePerformanceMeasures(predictions, labels, prediction_probability):
    
    precision = precision_score(y_true = labels, y_pred = predictions)
    recall    = recall_score(y_true = labels, y_pred = predictions)
    roc_score = roc_auc_score(labels, prediction_probability)
    accuracy  = accuracy_score(y_true = labels, y_pred = predictions)
    f_measure = 2*(precision * recall)/float(precision + recall) 
    g_mean = math.sqrt(precision * recall)
    
    metrics = [precision, recall, roc_score, accuracy, f_measure, g_mean]
    
    return metrics

In [2]:
def computePerformanceMeasuresForTechniques():
    annotated_directory = 'dataset/annotated/'
    resultsFile = open('dataset/performanceMeasures.csv','w+')
    for projectName in os.listdir(annotated_directory):
        project = pd.read_csv(annotated_directory+projectName)
        for techniques in ['Accuracy_Voting','FMeasure_Voting','GMean_Voting','Accuracy_RF','FMeasure_RF','GMean_RF','Accuracy_Ada','FMeasure_Ada','GMean_Ada']:
            prediction = project[techniques+'_Pred']
            labels     = project['buggy']
            prediction_probability = project[techniques+'_Pred_Prob']
            measures = (','.join(str(x) for x in computePerformanceMeasures(prediction,labels,prediction_probability))+'\n')
            resultsFile.write(str(projectName)+','+techniques+','+measures)


In [5]:
computePerformanceMeasuresForTechniques()

In [1]:
for strings in ['Voting','RF','Ada']:
        print strings

Voting
RF
Ada


In [50]:
import pandas as pd
for fileName in os.listdir('dataset/annotated'):
        np.where(arr == 15)

Index([u'wmc', u'dit', u'noc', u'cbo', u'rfc', u'lcom', u'ca', u'ce', u'npm',
       u'lcom3', u'loc', u'dam', u'moa', u'mfa', u'cam', u'ic', u'cbm', u'amc',
       u'max_cc', u'avg_cc', u'buggy', u'Accuracy_Voting_Pred',
       u'Accuracy_Voting_Pred_Prob', u'FMeasure_Voting_Pred',
       u'FMeasure_Voting_Pred_Prob', u'GMean_Voting_Pred',
       u'GMean_Voting_Pred_Prob', u'Accuracy_RF_Pred',
       u'Accuracy_RF_Pred_Prob', u'FMeasure_RF_Pred', u'FMeasure_RF_Pred_Prob',
       u'GMean_RF_Pred', u'GMean_RF_Pred_Prob', u'Accuracy_Ada_Pred',
       u'Accuracy_Ada_Pred_Prob', u'FMeasure_Ada_Pred',
       u'FMeasure_Ada_Pred_Prob', u'GMean_Ada_Pred', u'GMean_Ada_Pred_Prob',
       u'Function to be selected'],
      dtype='object')
Index([u'wmc', u'dit', u'noc', u'cbo', u'rfc', u'lcom', u'ca', u'ce', u'npm',
       u'lcom3', u'loc', u'dam', u'moa', u'mfa', u'cam', u'ic', u'cbm', u'amc',
       u'max_cc', u'avg_cc', u'buggy', u'Accuracy_Voting_Pred',
       u'Accuracy_Voting_Pred_Prob', u'

Index([u'wmc', u'dit', u'noc', u'cbo', u'rfc', u'lcom', u'ca', u'ce', u'npm',
       u'lcom3', u'loc', u'dam', u'moa', u'mfa', u'cam', u'ic', u'cbm', u'amc',
       u'max_cc', u'avg_cc', u'buggy', u'Accuracy_Voting_Pred',
       u'Accuracy_Voting_Pred_Prob', u'FMeasure_Voting_Pred',
       u'FMeasure_Voting_Pred_Prob', u'GMean_Voting_Pred',
       u'GMean_Voting_Pred_Prob', u'Accuracy_RF_Pred',
       u'Accuracy_RF_Pred_Prob', u'FMeasure_RF_Pred', u'FMeasure_RF_Pred_Prob',
       u'GMean_RF_Pred', u'GMean_RF_Pred_Prob', u'Accuracy_Ada_Pred',
       u'Accuracy_Ada_Pred_Prob', u'FMeasure_Ada_Pred',
       u'FMeasure_Ada_Pred_Prob', u'GMean_Ada_Pred', u'GMean_Ada_Pred_Prob',
       u'Function to be selected'],
      dtype='object')
Index([u'wmc', u'dit', u'noc', u'cbo', u'rfc', u'lcom', u'ca', u'ce', u'npm',
       u'lcom3', u'loc', u'dam', u'moa', u'mfa', u'cam', u'ic', u'cbm', u'amc',
       u'max_cc', u'avg_cc', u'buggy', u'Accuracy_Voting_Pred',
       u'Accuracy_Voting_Pred_Prob', u'

In [13]:
print xx

Index([u'Accuracy_Voting_Pred', u'Accuracy_Voting_Pred_Prob',
       u'FMeasure_Voting_Pred', u'FMeasure_Voting_Pred_Prob',
       u'GMean_Voting_Pred', u'GMean_Voting_Pred_Prob', u'Accuracy_RF_Pred',
       u'Accuracy_RF_Pred_Prob', u'FMeasure_RF_Pred', u'FMeasure_RF_Pred_Prob',
       u'GMean_RF_Pred', u'GMean_RF_Pred_Prob', u'Accuracy_Ada_Pred',
       u'Accuracy_Ada_Pred_Prob', u'FMeasure_Ada_Pred',
       u'FMeasure_Ada_Pred_Prob', u'GMean_Ada_Pred', u'GMean_Ada_Pred_Prob'],
      dtype='object')


In [33]:
for fileName in os.listdir('dataset/DSF'):
    fil = pd.read_csv('dataset/DSF/'+fileName)
    fil=fil.drop(columns=tt)
    fil.to_csv('dataset/finalResults/'+fileName,index=False)

In [10]:
fil = pd.read_csv('dataset/DSF/camel-1.6.csv') 
tt=fil.columns.values
tt= (tt[21:39])

In [13]:
print (len(tt))

18
